In [155]:
import pandas as pd
import numpy as np
import librosa
import soundfile
from datasets import load_dataset, load_from_disk, DatasetDict
from glob import glob
import os
from tqdm import tqdm
tqdm.pandas()
import sys
sys.path.append('/Users/markjos/projects/malachor5/scripts')
from asr_dataset import clip_segment, validate_clipfile

In [129]:
sli_csv='/Users/markjos/projects/malachor5/data/tira-snippets-30s-uncondensed/tira-cm-snip-sli.csv'

In [130]:
sli_df=pd.read_csv(sli_csv)

In [131]:
sli_df.head()

,label,audio_path,output
0,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s17.0ms...,1
1,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s20.0ms...,0
2,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s22.0ms...,0
3,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s25.0ms...,1
4,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s25.0ms...,1


## Reintroduce columns
Merge `sli_df` with `snippets_df`

In [132]:
snippets_data='/Users/markjos/projects/malachor5/data/tira-snippets-30s-uncondensed/metadata.csv'
snippets_df=pd.read_csv(snippets_data)
snippets_df=snippets_df[snippets_df['mother_split']=='train']
snippets_df.head()

,file_name,start,end,turn_label,mother_old_start,mother_old_end,mother_transcription,mother_eaf_source,mother_wav_source,mother_raw_transcription,...,mother_end_list,mother_trans_list,mother_pairwise_cos_sim,mother_vad_timestamps,mother_tira_timestamps,mother_merged_timestamps,mother_merged_labels,mother_timestamps_30s,mother_labels_30s,mother_turns_30s
0,/Users/markjos/projects/malachor5/data/tira-sn...,204.440,205.807,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."
1,/Users/markjos/projects/malachor5/data/tira-sn...,207.275,208.777,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."
2,/Users/markjos/projects/malachor5/data/tira-sn...,209.705,211.561,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."
3,/Users/markjos/projects/malachor5/data/tira-sn...,212.034,212.354,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."
4,/Users/markjos/projects/malachor5/data/tira-sn...,212.675,215.173,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."


In [133]:
snippets_df['audio_path']=snippets_df['file_name'].apply(os.path.basename)
merge_df=pd.merge(sli_df, snippets_df, on='audio_path')
merge_df.head()

,label,audio_path,output,file_name,start,end,turn_label,mother_old_start,mother_old_end,mother_transcription,...,mother_end_list,mother_trans_list,mother_pairwise_cos_sim,mother_vad_timestamps,mother_tira_timestamps,mother_merged_timestamps,mother_merged_labels,mother_timestamps_30s,mother_labels_30s,mother_turns_30s
0,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s17.0ms...,1,/Users/markjos/projects/malachor5/data/tira-sn...,204.440,205.807,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."
1,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s20.0ms...,0,/Users/markjos/projects/malachor5/data/tira-sn...,207.275,208.777,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."
2,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s22.0ms...,0,/Users/markjos/projects/malachor5/data/tira-sn...,209.705,211.561,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."
3,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s25.0ms...,1,/Users/markjos/projects/malachor5/data/tira-sn...,212.034,212.354,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."
4,NaN,HH01082021-m03s07ms011-m04s09ms552-m0.0s25.0ms...,1,/Users/markjos/projects/malachor5/data/tira-sn...,212.675,215.173,VAD,217011,219552,àprí jɜ̀dí ðáŋàlà,...,"[219552.0, 225023.0]","['àprí jɜ̀dí ðáŋàlà', 'àprí jɜ̀dí ðá...",0.659426,"[(195.19299999999998, 195.834, 196.475), (198....","[(217.011, 218.2815, 219.552), (221.371, 223.1...","[(195.19299999999998, 195.834, 196.475), (198....","['VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'VA...","((204.44, 205.12349999999998, 205.807), (207.2...","('VAD', 'VAD', 'VAD', 'VAD', 'VAD', 'àprí jɜ...","[(204.44, 205.12349999999998, 205.807, 'VAD'),..."


In [134]:
merge_df.shape, sli_df.shape, snippets_df.shape

((43079, 50), (43088, 3), (43079, 48))

In [135]:
sli_df=merge_df

It seems 9 records have been consumed by the abyss. ¯\\_(ツ)_/¯

Let's add a column for the index of a snippet within a clip

In [136]:
sli_df['mother_start_list']=sli_df['mother_start_list'].apply(eval)

In [137]:
sli_df['mother_start_list']=sli_df['mother_start_list'].apply(lambda l:[int(x) for x in l])

In [138]:
sli_df['clip_i']=0
def set_clip_indices(mother_clip):
    brothers=sli_df[sli_df['mother_path']==mother_clip].copy()
    brothers=brothers.sort_values('start')
    sli_df.loc[brothers.index, 'clip_i']=list(range(len(brothers)))
    
pd.Series(sli_df['mother_path'].unique()).apply(set_clip_indices)
sli_df.sort_values(['mother_path','clip_i'])[['start', 'mother_path', 'clip_i']].iloc[:20]

,start,mother_path,clip_i
0,204.440,HH01082021-m03s07ms011-m04s09ms552.wav,0
1,207.275,HH01082021-m03s07ms011-m04s09ms552.wav,1
2,209.705,HH01082021-m03s07ms011-m04s09ms552.wav,2
3,212.034,HH01082021-m03s07ms011-m04s09ms552.wav,3
4,212.675,HH01082021-m03s07ms011-m04s09ms552.wav,4
5,217.011,HH01082021-m03s07ms011-m04s09ms552.wav,5
6,221.371,HH01082021-m03s07ms011-m04s09ms552.wav,6
7,227.154,HH01082021-m03s07ms011-m04s09ms552.wav,7
8,229.837,HH01082021-m03s07ms011-m04s09ms552.wav,8
9,271.910,HH01082021-m04s13ms401-m05s16ms528.wav,0


Now let's condense English records using a while loop

Rip numpy array operations :(

In [139]:
def condense_records(mother_path):
    record_df=sli_df[sli_df['mother_path']==mother_path]
    condensed_rows=[]
    i=0
    while i<len(record_df):
        row=record_df.iloc[i]
        start=row['start']
        start_i=row['clip_i']
        end_i=start_i
        i+=1
        if row['output']!=1:
            condensed_rows.append(row)
            continue
        while i<len(record_df) and record_df.iloc[i]['output']==1:
            row=record_df.iloc[i]
            end_i=row['end']
            i+=1
        new_row=row.copy()
        new_row['start']=start
        new_row['clip_i']=(start_i,end_i)
        condensed_rows.append(new_row)
    return condensed_rows

first_clip = sli_df['mother_path'][900]
sli_df[sli_df['mother_path']==first_clip][['start', 'end', 'output']],\
pd.DataFrame(condense_records(first_clip))[['start', 'end', 'output']]

(       start      end  output
 895  309.779  310.572       1
 896  310.994  312.192       1
 897  313.610  316.495       1
 898  317.407  318.014       1
 899  318.554  319.010       0
 900  319.583  321.372       1
 901  321.727  322.604       1
 902  323.920  324.241       1
 903  325.050  326.477       0
 904  329.169  331.302       0
 905  334.413  336.899       0,
        start      end  output
 898  309.779  318.014       1
 899  318.554  319.010       0
 902  319.583  324.241       1
 903  325.050  326.477       0
 904  329.169  331.302       0
 905  334.413  336.899       0)

In [140]:
records=[]
for clip_path in tqdm(sli_df['mother_path'].unique()):
    records.extend(condense_records(clip_path))
condensed_df=pd.DataFrame(records)

100%|██████████| 4421/4421 [00:20<00:00, 214.10it/s]


In [141]:
(condensed_df['output']==0).value_counts(), (sli_df['output']==0).value_counts()

(output
 True     26063
 False     9162
 Name: count, dtype: int64,
 output
 True     26063
 False    17016
 Name: count, dtype: int64)

In [142]:
condensed_df.columns

Index(['label', 'audio_path', 'output', 'file_name', 'start', 'end',
       'turn_label', 'mother_old_start', 'mother_old_end',
       'mother_transcription', 'mother_eaf_source', 'mother_wav_source',
       'mother_raw_transcription', 'mother_clip', 'mother_wav_rawpath',
       'mother_path', 'mother_allosaurus', 'mother_clap_ipa_cos_sim',
       'mother_wada_snr', 'mother_nist_stnr', 'mother_speaker-diarization-3.1',
       'mother_voice-activity-detection', 'mother_whisper-large-v3',
       'mother_clapipa-transcription-allosaurus', 'mother_vad_s',
       'mother_drz_s', 'mother_trans_len', 'mother_pcnt_speech',
       'mother_trans_len_sq', 'mother_trans_len_log', 'mother_cos_sim_softmax',
       'mother_cos_sim_log', 'mother_duration', 'mother_split', 'mother_start',
       'mother_end', 'mother_overlap', 'mother_num_overlap',
       'mother_overlap_split', 'mother_start_list', 'mother_end_list',
       'mother_trans_list', 'mother_pairwise_cos_sim', 'mother_vad_timestamps',
     

In [143]:
condensed_df['mother_start']

0         187011
1         187011
2         187011
4         187011
5         187011
          ...   
43073    1892560
43074    1892560
43075    1892560
43077    1892560
43078    1892560
Name: mother_start, Length: 35225, dtype: int64

In [146]:
snippet_dir='/Users/markjos/projects/malachor5/data/tira-snippets-30s/clips'

def make_clips_for_record(row, target_dir=snippet_dir):
    clip=row['mother_path']
    split = row['mother_split']
    clip_fullpath=os.path.join('/Users/markjos/projects/malachor5/data/tira-code-mixed/clips', split, clip)
    lang='tira' if row['output']==0 else 'eng'
    wav_source=row['mother_wav_source']
    record_start=row['mother_start']
    wav, _= librosa.load(clip_fullpath, sr=16_000, mono=True)
    start=int(row['start']*1_000)-record_start
    end=int(row['end']*1_000)-record_start
    new_clip=clip_segment(
        wav=wav,
        start_ms=start,
        end_ms=end,
        wav_basename=os.path.basename(wav_source),
        target_dir=os.path.join(target_dir, lang),
    )
    return new_clip

make_clips_for_record(condensed_df.iloc[0])

'/Users/markjos/projects/malachor5/data/tira-snippets-30s/clips/eng/HH01082021-m00s17ms429-m00s18ms796.wav'

In [147]:
condensed_df['file_name_condensed']=condensed_df.progress_apply(make_clips_for_record, axis=1)

  0%|          | 0/35225 [00:00<?, ?it/s]

100%|██████████| 35225/35225 [00:01<00:00, 26509.28it/s]


In [148]:
snippets_condensed='/Users/markjos/projects/malachor5/data/tira-snippets-30s'
condensed_df['file_name_condensed']=condensed_df['file_name_condensed'].apply(
    lambda fp: os.path.relpath(fp, snippets_condensed)
)
metadata='/Users/markjos/projects/malachor5/data/tira-snippets-30s/metadata.csv'
condensed_df.rename({'file_name': 'file_name_uncondensed', 'file_name_condensed': 'file_name'},axis=1).to_csv(metadata, index=False)

In [149]:
metadata=pd.read_csv(metadata)
metadata['file_name'][0]

'clips/eng/HH01082021-m00s17ms429-m00s18ms796.wav'

In [150]:
condensed_df['file_name_condensed'][0]

'clips/eng/HH01082021-m00s17ms429-m00s18ms796.wav'

In [151]:
ds=load_dataset('audiofolder', data_dir=snippets_condensed)
ds

Resolving data files: 100%|██████████| 34936/34936 [00:00<00:00, 62399.77it/s] 
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 34935 examples [00:21, 1593.91 examples/s]


DatasetDict({
    train: Dataset({
        features: ['audio', 'label', 'audio_path', 'output', 'file_name_uncondensed', 'start', 'end', 'turn_label', 'mother_old_start', 'mother_old_end', 'mother_transcription', 'mother_eaf_source', 'mother_wav_source', 'mother_raw_transcription', 'mother_clip', 'mother_wav_rawpath', 'mother_path', 'mother_allosaurus', 'mother_clap_ipa_cos_sim', 'mother_wada_snr', 'mother_nist_stnr', 'mother_speaker-diarization-3.1', 'mother_voice-activity-detection', 'mother_whisper-large-v3', 'mother_clapipa-transcription-allosaurus', 'mother_vad_s', 'mother_drz_s', 'mother_trans_len', 'mother_pcnt_speech', 'mother_trans_len_sq', 'mother_trans_len_log', 'mother_cos_sim_softmax', 'mother_cos_sim_log', 'mother_duration', 'mother_split', 'mother_start', 'mother_end', 'mother_overlap', 'mother_num_overlap', 'mother_overlap_split', 'mother_start_list', 'mother_end_list', 'mother_trans_list', 'mother_pairwise_cos_sim', 'mother_vad_timestamps', 'mother_tira_timestamps', 'm

In [153]:
ds_lang_split=DatasetDict()
ds_lang_split['eng']=ds.filter(lambda r:r['output']==1)
ds_lang_split['tira']=ds.filter(lambda r:r['output']==0)
ds_lang_split

Filter: 100%|██████████| 34935/34935 [00:43<00:00, 799.11 examples/s]


DatasetDict({
    eng: DatasetDict({
        train: Dataset({
            features: ['audio', 'label', 'audio_path', 'output', 'file_name_uncondensed', 'start', 'end', 'turn_label', 'mother_old_start', 'mother_old_end', 'mother_transcription', 'mother_eaf_source', 'mother_wav_source', 'mother_raw_transcription', 'mother_clip', 'mother_wav_rawpath', 'mother_path', 'mother_allosaurus', 'mother_clap_ipa_cos_sim', 'mother_wada_snr', 'mother_nist_stnr', 'mother_speaker-diarization-3.1', 'mother_voice-activity-detection', 'mother_whisper-large-v3', 'mother_clapipa-transcription-allosaurus', 'mother_vad_s', 'mother_drz_s', 'mother_trans_len', 'mother_pcnt_speech', 'mother_trans_len_sq', 'mother_trans_len_log', 'mother_cos_sim_softmax', 'mother_cos_sim_log', 'mother_duration', 'mother_split', 'mother_start', 'mother_end', 'mother_overlap', 'mother_num_overlap', 'mother_overlap_split', 'mother_start_list', 'mother_end_list', 'mother_trans_list', 'mother_pairwise_cos_sim', 'mother_vad_timestamps

In [154]:
ds_path='/Users/markjos/projects/malachor5/data/pyarrow-datasets/tira-snippets'
ds_lang_split.save_to_disk(ds_path)

Saving the dataset (4/4 shards): 100%|██████████| 25773/25773 [00:18<00:00, 1391.20 examples/s]


In [158]:
load_from_disk(ds_path+'/tira')

DatasetDict({
    train: Dataset({
        features: ['audio', 'label', 'audio_path', 'output', 'file_name_uncondensed', 'start', 'end', 'turn_label', 'mother_old_start', 'mother_old_end', 'mother_transcription', 'mother_eaf_source', 'mother_wav_source', 'mother_raw_transcription', 'mother_clip', 'mother_wav_rawpath', 'mother_path', 'mother_allosaurus', 'mother_clap_ipa_cos_sim', 'mother_wada_snr', 'mother_nist_stnr', 'mother_speaker-diarization-3.1', 'mother_voice-activity-detection', 'mother_whisper-large-v3', 'mother_clapipa-transcription-allosaurus', 'mother_vad_s', 'mother_drz_s', 'mother_trans_len', 'mother_pcnt_speech', 'mother_trans_len_sq', 'mother_trans_len_log', 'mother_cos_sim_softmax', 'mother_cos_sim_log', 'mother_duration', 'mother_split', 'mother_start', 'mother_end', 'mother_overlap', 'mother_num_overlap', 'mother_overlap_split', 'mother_start_list', 'mother_end_list', 'mother_trans_list', 'mother_pairwise_cos_sim', 'mother_vad_timestamps', 'mother_tira_timestamps', 'm